In [ ]:
import os
import pyodbc
import json

os.getcwd()

In [ ]:
with open('Products.json', encoding='utf-8-sig') as f:
    product_data = json.load(f)
    
with open('customer.json', encoding='utf-8-sig') as f:
    customer_data = json.load(f)
    
with open('orders.json', encoding='utf-8-sig') as f:
    orders_data = json.load(f)

In [ ]:
print(f"Length of Products: {len(product_data)}")
print(f"Length of Customers: {len(customer_data)}")
print(f"Length of Orders: {len(orders_data)}")

print("\n")
print(product_data[0])
print("\n")
print(customer_data[0])
print("\n")
print(orders_data[0])

In [ ]:
from gremlin_python.driver import client, serializer
from gremlin_python.driver.aiohttp.transport import AiohttpTransport


client = client.Client('wss://olist-graph-db.gremlin.cosmosdb.azure.com:443/','g', 
             username="/dbs/db-olist/colls/graph-olist",
             password="qDT6DYEtwXZyMkXB4qIF7GkeQ2q74Xw0RdIEom9mc074w22wpvcn6i0AZp2qlPYkUpcVmlrmTMpUACDbcLuIwg==",
             message_serializer=serializer.GraphSONSerializersV2d0(),
             transport_factory=lambda: AiohttpTransport(call_from_event_loop=True)
             )

for obj in customer_data:
    query = f"g.addV('customer').property('id', '{obj['customer_id']}').property('customer_unique_id', '{obj['customer_unique_id']}').property('geolocation_id', '{obj['geolocation_id']}').property('pk','pk')"
    client.submitAsync(query)


In [ ]:
for obj in product_data:
    query = (f"g.addV('product').property('id', '{obj.get('product_id')}')"
             f".property('product_name_length', '{obj.get('product_name_length') or -99}')"
             f".property('product_description_length', '{obj.get('product_description_length') or -99}')"
             f".property('product_photos_qty', '{obj.get('product_photos_qty') or -99}')"
             f".property('product_weight_g', '{obj.get('product_weight_g') or -99}')"
             f".property('product_length_cm', '{obj.get('product_length_cm') or -99}')"
             f".property('product_height_cm', '{obj.get('product_height_cm') or -99}')"
             f".property('product_width_cm', '{obj.get('product_width_cm') or -99}')"
             f".property('product_category_name', '{obj.get('product_category_name') or 'Unknown'}')"
             f".property('pk','pk')")

    client.submitAsync(query)

In [ ]:
for obj in orders_data:
    query = f"g.addV('order').property('id', '{obj['order_id']}') \
              .property('customer_id', '{obj.get('customer_id')}') \
              .property('order_status', '{obj.get('order_status') or 'Unknown'}') \
              .property('order_purchase_timestamp', '{obj.get('order_purchase_timestamp')}') \
              .property('order_approved_at', '{obj.get('order_approved_at')}') \
              .property('order_delivered_carrier_date', '{obj.get('order_delivered_carrier_date')}') \
              .property('order_delivered_customer_date', '{obj.get('order_delivered_customer_date')}') \
              .property('order_estimated_delivery_date', '{obj.get('order_estimated_delivery_date')}') \
              .property('pk','pk')"
    client.submit(query)

#g.V().hasLabel('order').has('order_id', '00010242fe8c5a6d1ba2dd792cb16214').addE('orderedBy').to(g.V().hasLabel('customer').has('customer_id', '3ce436f183e68e07877b285a838db11a'))